# Import Libraries

In [1]:
# install `jupyter-dash` and restart runtime
# https://github.com/plotly/jupyter-dash
!pip install jupyter-dash

     |████████████████████████████████| 7.3 MB 4.3 MB/s 
     |████████████████████████████████| 23.9 MB 13 kB/s 
     |████████████████████████████████| 357 kB 67.1 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=aaa4d556fe21b6321ea52db90c866c44864a450a6956c2d8f7f2709565bbf368
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=8e5592905490f4185c5959b33162802d6ab71e428674dd361b2b15bfb785f9e9
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=ca6794750900900c5c9fc66d4701e6723725440143e5994745c7072cc07b5c6a
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aefd

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  


# Download SpaceX Dataset

In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-06 18:04:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-09-06 18:04:14 (275 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [4]:
spacex_df = pd.read_csv("/content/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


# Create a dash application

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# create a dash application
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# create a list of all launch sites
launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites.insert(0, 'All Sites')

# payload min , max range
payload_min, payload_max = spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()
app.layout = html.Div(
    children=[html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36',
                                                                'font-size': 40}),
              
              # https://dash.plotly.com/dash-core-components/dropdown
              dcc.Dropdown(id='site-dropdown',
                           options=[{'label': i, 'value': i} for i in launch_sites],
                           value='All Sites',
                           placeholder="Select a Launch Site here",
                           searchable=True),
              html.Br(),

              html.Div(dcc.Graph(id='success-pie-chart')),
              html.Br(),

              html.P("Payload range (Kg):"),
            
              # https://dash.plotly.com/dash-core-components/rangeslider
              dcc.RangeSlider(id='payload-slider', min=payload_min, max=10000,
                              marks={
                                  0: '0',
                                  2500: '2500',
                                  5000: '5000',
                                  7500: '7500',
                                  10000: '10000'
                              },
                              step=1000, value=[payload_min, payload_max]),
              html.Br(),
              html.Div(dcc.Graph(id='success-payload-scatter-chart')),
              ])


@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):

    # https://community.plotly.com/t/graph-if-empty-dropdown/7483/2
    # leave callback Output unchanged
    if entered_site is None:
        raise Exception()

    # to clear the graph
    # if entered_site is None:
    #     return {'data': []}

    # https://plotly.com/python/pie-charts/
    if entered_site == 'All Sites':
        fig = px.pie(spacex_df, names="Launch Site", values='class', title='Total Success Launches By Site')
    else:
        pie_filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(pie_filtered_df, names="class", title='Total Success Launches By Site')
    return fig


@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value'), ]
)
def get_slider_chart(entered_site, payload_value):
    # https://plotly.com/python/line-and-scatter/
    if entered_site == 'All Sites':
        slide_df = spacex_df[(spacex_df['Payload Mass (kg)'].between(left=payload_value[0], right=payload_value[1]))]
        scatter_plot = px.scatter(slide_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                                  labels={
                                      "class": "Launch Outcome",
                                  },
                                  title="Correlation between Payload and Success for all sites")
    else:
        slide_df = spacex_df[(spacex_df['Payload Mass (kg)'].between(left=payload_value[0], right=payload_value[1])) & (
                spacex_df['Launch Site'] == entered_site)]
        scatter_plot = px.scatter(slide_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                                  labels={
                                      "class": "Launch Outcome",
                                  },
                                  title="Correlation between Payload and Success for all sites")
    return scatter_plot

# run the app
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)

<IPython.core.display.Javascript object>